In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv("../../../data/cleaned/multi_var_wind_hourly_cleaned.csv")
df = df.sort_values('datetime').reset_index(drop=True)
df['wind_speed'] = df['wind_speed'].astype(float)

In [12]:
df = df[['datetime', 'pressure', 'temperature', 'humidity', 'wind_speed', 'u', 'v']]


In [13]:
df.head()

,datetime,pressure,temperature,humidity,wind_speed,u,v
0,2024-01-01 00:00:00,758.466667,-1.016667,62.166667,0.100000,-0.022693,-0.047340
1,2024-01-01 01:00:00,758.383333,-1.533333,64.000000,0.283333,-0.113825,0.011167
2,2024-01-01 02:00:00,758.383333,-1.150000,61.333333,0.200000,-0.169286,-0.061781
3,2024-01-01 03:00:00,758.783333,-1.166667,58.666667,0.500000,0.142045,-0.397690
4,2024-01-01 04:00:00,759.000000,-1.483333,62.166667,0.966667,-0.512590,-0.609574


In [14]:
df['datetime'] = pd.to_datetime(df['datetime'])   # or df['datetime'] if that's the name

In [15]:
# now extract hour
df['hour'] = df['datetime'].dt.hour
df['sin_h'] = np.sin(2 * np.pi * df['hour'] / 24)
df['cos_h'] = np.cos(2 * np.pi * df['hour'] / 24)

In [16]:
df.head()

,datetime,pressure,temperature,humidity,wind_speed,u,v,hour,sin_h,cos_h
0,2024-01-01 00:00:00,758.466667,-1.016667,62.166667,0.100000,-0.022693,-0.047340,0,0.000000,1.000000
1,2024-01-01 01:00:00,758.383333,-1.533333,64.000000,0.283333,-0.113825,0.011167,1,0.258819,0.965926
2,2024-01-01 02:00:00,758.383333,-1.150000,61.333333,0.200000,-0.169286,-0.061781,2,0.500000,0.866025
3,2024-01-01 03:00:00,758.783333,-1.166667,58.666667,0.500000,0.142045,-0.397690,3,0.707107,0.707107
4,2024-01-01 04:00:00,759.000000,-1.483333,62.166667,0.966667,-0.512590,-0.609574,4,0.866025,0.500000


In [17]:
df = df.set_index('datetime')

In [18]:
df['month'] = df.index.month
df['season'] = df['month'] % 12 // 3 + 1  # 1=spring, 2=summer, etc.

In [19]:

# Create lag features (1-6 hours)
for i in range(1, 7):
    df[f'wind_speed_lag_{i}'] = df['wind_speed'].shift(i)
    df[f'temp_lag_{i}'] = df['temperature'].shift(i)
    df[f'humidity_lag_{i}'] = df['humidity'].shift(i)
    df[f'pressure_lag_{i}'] = df['pressure'].shift(i)
    df[f'u_lag_{i}'] = df['u'].shift(i)
    df[f'v_lag_{i}'] = df['v'].shift(i)

In [20]:

# Rolling statistics
df['wind_speed_3hr_avg'] = df['wind_speed'].rolling(window=3).mean()
df['wind_speed_6hr_std'] = df['wind_speed'].rolling(window=6).std()

In [21]:
# Drop rows with NaN values created by shifting
df = df.dropna()

In [22]:
# Define target for 1-hour forecasting
df['target'] = df['wind_speed'].shift(-1)
df = df.dropna()


In [23]:
# Chronological split
split_point = int(len(df) * 0.8)
train_df = df.iloc[:split_point]
test_df = df.iloc[split_point:]


In [24]:
# Prepare features and target
X_train = train_df.drop(['target', 'wind_speed'], axis=1)
y_train = train_df['target']
X_test = test_df.drop(['target', 'wind_speed'], axis=1)
y_test = test_df['target']

In [25]:
X_train.head()

,pressure,temperature,humidity,u,v,hour,sin_h,cos_h,month,season,...,u_lag_5,v_lag_5,wind_speed_lag_6,temp_lag_6,humidity_lag_6,pressure_lag_6,u_lag_6,v_lag_6,wind_speed_3hr_avg,wind_speed_6hr_std
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-01 06:00:00,759.466667,2.166667,50.000000,0.532663,1.072439,6,1.000000,6.123234e-17,1,1,...,-0.113825,0.011167,0.100000,-1.016667,62.166667,758.466667,-0.022693,-0.047340,1.016667,0.468409
2024-01-01 07:00:00,759.733333,3.566667,43.333333,-0.161001,0.264452,7,0.965926,-2.588190e-01,1,1,...,-0.169286,-0.061781,0.283333,-1.533333,64.000000,758.383333,-0.113825,0.011167,0.816667,0.455633
2024-01-01 08:00:00,759.533333,4.033333,46.666667,1.090914,1.377346,8,0.866025,-5.000000e-01,1,1,...,0.142045,-0.397690,0.200000,-1.150000,61.333333,758.383333,-0.169286,-0.061781,1.322222,0.679842
2024-01-01 09:00:00,759.100000,4.483333,46.000000,1.786727,1.452116,9,0.707107,-7.071068e-01,1,1,...,-0.512590,-0.609574,0.500000,-1.166667,58.666667,758.783333,0.142045,-0.397690,1.611111,0.800515
2024-01-01 10:00:00,758.833333,4.616667,45.666667,2.723018,1.936597,10,0.500000,-8.660254e-01,1,1,...,-0.623646,0.041307,0.966667,-1.483333,62.166667,759.000000,-0.512590,-0.609574,2.605556,1.120945


In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt
import joblib

# After completing Step 1 data preparation
# X_train, y_train, X_test, y_test are already prepared

# Define evaluation function for Bayesian optimization
def xgb_evaluate(max_depth, learning_rate, n_estimators, subsample, colsample_bytree, min_child_weight):
    # Convert parameters to appropriate types
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'min_child_weight': min_child_weight,
        'objective': 'reg:squarederror',
        'tree_method': 'hist',  # Faster training
        'random_state': 42
    }
    
    # Train model with these parameters
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_pred = model.predict(X_test)
    
    # Calculate negative RMSE (Bayesian optimization maximizes the function)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Negative because bayes_opt maximizes the function

# Define parameter search space
pbounds = {
    'max_depth': (3, 10),           # Tree depth
    'learning_rate': (0.01, 0.3),   # Learning rate
    'n_estimators': (100, 1000),    # Number of trees
    'subsample': (0.6, 1.0),        # Row sampling ratio
    'colsample_bytree': (0.6, 1.0), # Feature sampling ratio
    'min_child_weight': (1, 10)     # Minimum sum of instance weight
}

# Initialize Bayesian optimizer
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)


ModuleNotFoundError: No module named 'bayes_opt'

In [ ]:
# Run optimization (adjust init_points and n_iter based on computational resources)
print("Starting Bayesian optimization...")
optimizer.maximize(
    init_points=5,  # Initial random points
    n_iter=20       # Iterations of optimization
)

# Get best parameters
best_params = optimizer.max['params']
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['objective'] = 'reg:squarederror'
best_params['tree_method'] = 'hist'
best_params['random_state'] = 42

print("Best parameters found:")
print(best_params)


In [ ]:
# Train final model with optimized parameters
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X_train, y_train)


In [ ]:
# Save the trained model
joblib.dump(final_model, 'wind_speed_forecast_xgb-hpo_model.pkl')


In [ ]:
# Evaluate on test set
y_pred = final_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test R²: {r2:.4f}")


In [ ]:
# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 important features:")
print(feature_importance.head(10))